In [206]:
import gensim
import csv
import pandas as pd
import pdftotext
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(400)
import nltk
nltk.download('wordnet')

import warnings 
warnings.filterwarnings(action="ignore", message="^internal gelsd")

stemmer = SnowballStemmer("english")
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

# Tokenize and lemmatize
def preprocess(text):
    result=[]
    for token in gensim.utils.simple_preprocess(text) :
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
            
    return result

df=pd.read_csv("Targetnames.csv")
#df["Text"]=df
#df['Target']=1
#df = df.drop(df.columns[0], axis=1)
#df.tail()
#pdf[0]

# df.head()

processed_docs = []

def fileread(df):
    with open(df, "rb") as f:
        conv = pdftotext.PDF(f)
        text=conv[0]
        processedtext = preprocess(text)
    return processedtext


# with open('Filewrite.txt', 'w') as f:
#     f.write("\n\n".join(pdf))

# text[0]

for i in range(len(df)):
    dictfile = fileread(df.Path[i])
    processed_docs.append(dictfile)

#dictfile
#processed_docs[13]

# Appendframe = df
# # #Appendframe.tail()
# # for i in df.Text:
# Appendframe.Text[0]=Appendframe.append(Appendframe, ignore_index=True)
# #Appendframe = Appendframe.append(df, ignore_index=True)
# Appendframe.Text[0]
# ddf['Target']="COI"
# ddf['Text']=ddf['DATE (MM/DD/YYYY)']
# fileread(df.Path[13])
# processed_docs.to_csv("A.csv")
dictionary = gensim.corpora.Dictionary(processed_docs)
# dictionary[412]
bow_corpus =[dictionary.doc2bow(doc) for doc in processed_docs]
#bow_corpus[1187]
# document_num = 0
# bow_doc_x = bow_corpus[document_num]

# for i in range(len(bow_doc_x)):
#     print("Word {} (\"{}\") appears {} time.".format(bow_doc_x[i][0], 
#                                                      dictionary[bow_doc_x[i][0]], 
#                                                      bow_doc_x[i][1]))

lda_model =  gensim.models.LdaMulticore(bow_corpus, 
                                   num_topics = 14, 
                                   id2word = dictionary,                                    
                                   passes = 10,
                                   workers = 2)
lda_model.save('lda_train.model')

# for idx, topic in lda_model.print_topics(-1):
#     print("Topic: {} \nWords: {}".format(idx, topic ))
#     print("\n")

# from nltk.corpus import stopwords
# stop_words = stopwords.words('english')
# stop_words.extend(['come','order','try','go','get','make','drink','plate','dish','restaurant','place',
#                   'would','really','like','great','service','came','got'])

# def remove_stopwords(texts):
#     return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

# def bigrams(words, bi_min=15, tri_min=10):
#     bigram = gensim.models.Phrases(words, min_count = bi_min)
#     bigram_mod = gensim.models.phrases.Phraser(bigram)
#     return bigram_mod
    
# def get_corpus(df):
#     #df['Text'] = strip_newline(df.Path)
#     #words = list(df)
#     words = remove_stopwords(df)
#     bigram_mod = bigrams(words)
#     bigram = [bigram_mod[review] for review in words]
#     id2word = gensim.corpora.Dictionary(bigram)
#     id2word.filter_extremes(no_below=10, no_above=0.35)
#     id2word.compactify()
#     corpus = [id2word.doc2bow(text) for text in bigram]
#     return corpus, id2word, bigram

# train_corpus, train_id2word, bigram_train = get_corpus(processed_docs)

# import gensim
# import logging # This allows for seeing if the model converges. A log file is created.
# logging.basicConfig(filename='lda_model.log', format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

# with warnings.catch_warnings():
#     warnings.simplefilter('ignore')
#     lda_train = gensim.models.ldamulticore.LdaMulticore(
#                            corpus=train_corpus,
#                            num_topics=1,
#                            id2word=train_id2word,
#                            chunksize=100,
#                            workers=7, # Num. Processing Cores - 1
#                            passes=50,
#                            eval_every = 1,
#                            per_word_topics=True)
#     lda_train.save('lda_train.model')

lda_model.print_topics(20,num_words=15)[:2]

train_vecs = []
for i in range(len(df)):
    top_topics = lda_model.get_document_topics(bow_corpus[i], minimum_probability=0.0)
    topic_vec = [top_topics[i][1] for i in range(14)]
    #topic_vec.extend([df.iloc[i].real_counts]) # counts of reviews for restaurant
    #topic_vec.extend([len(df.iloc[i].text)]) # length review
    train_vecs.append(topic_vec)
    
#train_vecs[13]
# df.Target

X = np.array(train_vecs)
y = np.array(df.Target)

kf = KFold(2, shuffle=True, random_state=42)
cv_lr_f1, cv_lrsgd_f1, cv_svcsgd_f1,  = [], [], []

for train_ind, val_ind in kf.split(X, y):
    # Assign CV IDX
    X_train, y_train = X[train_ind], y[train_ind]
    X_val, y_val = X[val_ind], y[val_ind]
    
    # Scale Data
    scaler = StandardScaler()
    X_train_scale = scaler.fit_transform(X_train)
    X_val_scale = scaler.transform(X_val)

    # Logisitic Regression
    lr = LogisticRegression(
        class_weight= 'balanced',
        solver='newton-cg',
        fit_intercept=True
    ).fit(X_train_scale, y_train)

    y_pred = lr.predict(X_val_scale)
    cv_lr_f1.append(f1_score(y_val, y_pred, average='macro'))
    
    # Logistic Regression SGD
    sgd = linear_model.SGDClassifier(
        max_iter=1000,
        tol=1e-3,
        loss='log',
        class_weight='balanced'
    ).fit(X_train_scale, y_train)
    
    y_pred = sgd.predict(X_val_scale)
    cv_lrsgd_f1.append(f1_score(y_val, y_pred, average='macro'))
    
    # SGD Modified Huber
    sgd_huber = linear_model.SGDClassifier(
        max_iter=1000,
        tol=1e-3,
        alpha=20,
        loss='modified_huber',
        class_weight='balanced'
    ).fit(X_train_scale, y_train)
    
    y_pred = sgd_huber.predict(X_val_scale)
    cv_svcsgd_f1.append(f1_score(y_val, y_pred, average='macro'))

# print(f'Logistic Regression Val f1: {np.mean(cv_lr_f1):.3f} +- {np.std(cv_lr_f1):.3f}')
# print(f'Logisitic Regression SGD Val f1: {np.mean(cv_lrsgd_f1):.3f} +- {np.std(cv_lrsgd_f1):.3f}')
# print(f'SVM Huber Val f1: {np.mean(cv_svcsgd_f1):.3f} +- {np.std(cv_svcsgd_f1):.3f}')



#TESTING STARTS HERE
dftest=pd.read_csv("Testnames.csv")
testfile = []
for i in range(len(dftest)):
    dictfile = fileread(dftest.Path[i])
    testfile.append(dictfile)
    

#test['Text']=test
#test['Target']="COI"
#test = test.drop(test.columns[0], axis=1)
#test.tail()
#testfile[0]
test_corpus = [dictionary.doc2bow(doc) for doc in testfile]
#test_corpus[0]
##Testing

# def get_bigram(df):
#     """
#     For the test data we only need the bigram data built on 2017 reviews,
#     as we'll use the 2016 id2word mappings. This is a requirement due to 
#     the shapes Gensim functions expect in the test-vector transformation below.
#     With both these in hand, we can make the test corpus.
#     """
#     #df['text'] = strip_newline(df.text)
#     words = list(df)
#     words = remove_stopwords(words)
#     bigram = bigrams(words)
#     bigram = [bigram[review] for review in words]
#     return bigram

  
# bigram_test = get_bigram(test.Text)

# test_corpus = [train_id2word.doc2bow(text) for text in bigram_test]

test_vecs = []
for i in range(len(dftest)):
    top_topics = lda_model.get_document_topics(test_corpus[i], minimum_probability=0.0)
    topic_vec = [top_topics[i][1] for i in range(14)]
    #topic_vec.extend([df.iloc[i].real_counts]) # counts of reviews for restaurant
    #topic_vec.extend([len(df.iloc[i].text)]) # length review
    test_vecs.append(topic_vec)
    
test_vecs
W=np.array(test_vecs)
test_val_scale = scaler.transform(W)



[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/dhananjayneelakantan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Logistic Regression Val f1: 0.670 +- 0.081
Logisitic Regression SGD Val f1: 0.622 +- 0.033
SVM Huber Val f1: 0.610 +- 0.021


/Users/dhananjayneelakantan/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/Users/dhananjayneelakantan/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/dhananjayneelakantan/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/dhananjayneelakantan/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this

In [207]:
#Prediction
print(sgd.predict(test_val_scale), lr.predict(test_val_scale),sgd_huber.predict(test_val_scale))

[4] [4] [4]
